# Improved Belief Model - Colab Training Notebook

This notebook is set up to train the Improved Belief Model on Google Colab with optional GPU acceleration.

What it does:
- Installs dependencies
- Loads your project into Colab (upload zip or mount Drive)
- Parses existing logs into training states
- Trains the improved belief model (with configurable epochs/sample fraction)
- Saves the trained weights back to Drive or local runtime

Notes:
- If using GPU: Runtime → Change runtime type → Hardware accelerator: GPU.
- Ensure your logs exist (e.g., `logs/game28/mcts_games` and/or `logs/improved_games`).


In [ ]:
# Install system and Python dependencies
pip -q install --upgrade pip
pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
pip -q install tqdm numpy pyyaml


In [ ]:
#@title Mount Google Drive (optional) and set up project
USE_DRIVE = True  #@param {type:"boolean"}
PROJECT_DIR = "/content/28bot"  #@param {type:"string"}

import os, shutil
from pathlib import Path

if USE_DRIVE:
  from google.colab import drive
  drive.mount('/content/drive')
  DRIVE_BASE = "/content/drive/MyDrive"
  if not os.path.exists(f"{DRIVE_BASE}/28bot"):
    os.makedirs(f"{DRIVE_BASE}/28bot", exist_ok=True)
  PROJECT_DIR = f"{DRIVE_BASE}/28bot"

print("Project dir:", PROJECT_DIR)
Path(PROJECT_DIR).mkdir(parents=True, exist_ok=True)


In [ ]:
#@title Upload repo zip or clone (choose one)
MODE = "upload_zip"  #@param ["upload_zip", "git_clone"]
GIT_URL = ""  #@param {type:"string"}

import os, zipfile
from google.colab import files

os.chdir(PROJECT_DIR)

if MODE == "upload_zip":
  print("Please upload a zip containing the 28bot_v2 folder and logs/... folders")
  uploaded = files.upload()
  for fn in uploaded:
    if fn.endswith('.zip'):
      with zipfile.ZipFile(fn, 'r') as z:
        z.extractall(PROJECT_DIR)
      print("Unzipped:", fn)
else:
  if not GIT_URL:
    raise ValueError("Provide a GIT_URL or use upload_zip mode")
  # Use shell command via python to avoid lint in this cell
  import subprocess
  subprocess.run(["bash", "-lc", f"git clone {GIT_URL} {PROJECT_DIR}"])

print("Project contents:", os.listdir(PROJECT_DIR)[:20])


In [ ]:
#@title Configure training
EPOCHS = 20  #@param {type:"integer"}
SAMPLE_FRACTION = 0.5  #@param {type:"number"}
LR = 0.001  #@param {type:"number"}
TRUMP_ONLY = True  #@param {type:"boolean"}

import random
random.seed(42)


#@title Configure training
EPOCHS = 20  #@param {type:"integer"}
SAMPLE_FRACTION = 0.5  #@param {type:"number"}
LR = 0.001  #@param {type:"number"}
TRUMP_ONLY = True  #@param {type:"boolean"}

import random
random.seed(42)


In [ ]:
#@title Train on GPU with AMP (reads logs/ and saves models/)
import os, sys, json, math
from pathlib import Path
import torch
from torch import nn

# Add project to path
sys.path.insert(0, PROJECT_DIR)

from belief_model.improved_belief_net import create_improved_belief_model, train_improved_belief_model
from belief_model.advanced_parser import extract_all_game_states
from train_improved_belief import build_training_data_from_logs

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Load parsed states
dirs = [
    os.path.join(PROJECT_DIR, "logs", "game28", "mcts_games"),
    os.path.join(PROJECT_DIR, "logs", "improved_games")
]
parsed = extract_all_game_states(dirs)
print("Total parsed states:", len(parsed))

# Sample subset if requested
if SAMPLE_FRACTION < 1.0:
  k = max(1, int(len(parsed) * SAMPLE_FRACTION))
  import random as _r
  parsed = _r.sample(parsed, k)
  print("Sampled states:", len(parsed))

# Build training tuples
training = build_training_data_from_logs(parsed)
print("Training tuples:", len(training))

# Create model
model = create_improved_belief_model().to(device)

# Optional: trump-only loss wrapper
class TrumpOnlyWrapper(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, game_state, player_id):
    return self.model(game_state, player_id)

# Minimal trainer with AMP for trump-only
def train_trump_only(model, data, epochs=EPOCHS, lr=LR):
  opt = torch.optim.Adam(model.parameters(), lr=lr)
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())
  bce = nn.BCELoss()
  model.train()
  for ep in range(epochs):
    total = 0.0
    for game_state, pid, targets in data:
      opt.zero_grad(set_to_none=True)
      with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
        out = model(game_state, pid)
        loss = 0.0
        if 'trump' in targets:
          target_trump = torch.tensor(targets['trump'], dtype=torch.float32, device=device)
          loss = loss + bce(out.trump_suit.to(device), target_trump)
      scaler.scale(loss).backward()
      scaler.step(opt)
      scaler.update()
      total += float(loss.item())
    if (ep % 5) == 0:
      print(f"Epoch {ep}: loss={total/len(data):.4f}")
  return model

if TRUMP_ONLY:
  model = train_trump_only(model, training, epochs=EPOCHS, lr=LR)
else:
  # Fallback to full trainer (CPU-style) if desired
  model = train_improved_belief_model(model, training, epochs=EPOCHS, learning_rate=LR)

# Save
out_dir = os.path.join(PROJECT_DIR, "models")
Path(out_dir).mkdir(parents=True, exist_ok=True)
torch.save(model.state_dict(), os.path.join(out_dir, "improved_belief_model.pt"))
print("Saved:", os.path.join(out_dir, "improved_belief_model.pt"))


In [ ]:
#@title Configure training
EPOCHS = 20  #@param {type:"integer"}
SAMPLE_FRACTION = 0.5  #@param {type:"number"}
LR = 0.001  #@param {type:"number"}
TRUMP_ONLY = True  #@param {type:"boolean"}

import random
random.seed(42)


#@title Configure training
EPOCHS = 20  #@param {type:"integer"}
SAMPLE_FRACTION = 0.5  #@param {type:"number"}
LR = 0.001  #@param {type:"number"}
TRUMP_ONLY = True  #@param {type:"boolean"}

import random
random.seed(42)
